<center><div style="direction:rtl;font-family:B Lotus, B Nazanin, Tahoma">به نام خدا</div></center>
<h1><center><div style="direction:rtl;font-family:B Lotus, B Nazanin, Tahoma"> استخراج ویژگی با یک شبکه عمیق از قبل آموزش داده شده</div></center></h1>

In [2]:
import keras
import cv2
import os
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

Using TensorFlow backend.


### Load Trained Model

In [3]:
conv_base = VGG16(weights='imagenet',
                  include_top=False,#Remove FC and softmax
                  input_shape=(224, 224, 3))

In [4]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [5]:
conv_base.input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [6]:
conv_base.output

<tf.Tensor 'block5_pool/MaxPool:0' shape=(None, 7, 7, 512) dtype=float32>

### Create Classes Folder

In [7]:
base_dir = os.path.expanduser("./hand_dataset")
os.makedirs(base_dir, exist_ok=True)
base_dir

'./hand_dataset'

### Capture samples of classes and create samples' folder

In [20]:
cls ="kosar"
c = 0
path = os.path.join(base_dir, cls)
os.makedirs(path, exist_ok=True)

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret: 
      print(" No Image") 
      break
    frame = cv2.resize(frame, (224, 224))
    
    cv2.imshow('Webcam', frame)
    key = cv2.waitKey(1)
    if  key == 13: #13 is the Enter Key
        break
    elif key == 32: #32 is the Space Key
        c += 1
        image_path = os.path.join(path, str(c)+".png")
        print("save to: ", image_path)
        cv2.imwrite(image_path, frame)

cap.release()
cv2.destroyAllWindows()     

save to:  ./hand_dataset\kosar\1.png
save to:  ./hand_dataset\kosar\2.png
save to:  ./hand_dataset\kosar\3.png
save to:  ./hand_dataset\kosar\4.png
save to:  ./hand_dataset\kosar\5.png
save to:  ./hand_dataset\kosar\6.png
save to:  ./hand_dataset\kosar\7.png
save to:  ./hand_dataset\kosar\8.png
save to:  ./hand_dataset\kosar\9.png
save to:  ./hand_dataset\kosar\10.png
save to:  ./hand_dataset\kosar\11.png
save to:  ./hand_dataset\kosar\12.png
save to:  ./hand_dataset\kosar\13.png
save to:  ./hand_dataset\kosar\14.png
save to:  ./hand_dataset\kosar\15.png
save to:  ./hand_dataset\kosar\16.png
save to:  ./hand_dataset\kosar\17.png
save to:  ./hand_dataset\kosar\18.png
save to:  ./hand_dataset\kosar\19.png
save to:  ./hand_dataset\kosar\20.png
save to:  ./hand_dataset\kosar\21.png
save to:  ./hand_dataset\kosar\22.png
save to:  ./hand_dataset\kosar\23.png
save to:  ./hand_dataset\kosar\24.png
save to:  ./hand_dataset\kosar\25.png
save to:  ./hand_dataset\kosar\26.png
save to:  ./hand_data

### Total number of samples

In [21]:
total = 0
for root, dirs, files in os.walk("./hand_dataset"):
    total += len(files)
print("total files in a directory: ", total)

total files in a directory:  76


### Feature extraction of samples with trained model

In [9]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator


datagen = ImageDataGenerator(rescale=1./255)
batch_size = 25

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 7, 7, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels, generator.class_indices



### Train and lable features that exctract from trained model

In [22]:
train_features, train_labels, dictionary = extract_features(base_dir, total)

Found 76 images belonging to 3 classes.


In [11]:
train_features.shape

(48, 7, 7, 512)

In [12]:
train_labels.shape

(48,)

In [13]:
dictionary

{'cellphone': 0, 'mouse': 1}

### Flat the output of trained model(or input of classifier)

In [23]:
train_features = np.reshape(train_features, (-1, 7 * 7 * 512))

In [24]:
train_features.shape

(76, 25088)

### Simple Classifier

In [25]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train_features, train_labels) 


KNeighborsClassifier(n_neighbors=3)

In [26]:
def get_class_name(l):
    for name, label in dictionary.items():
        if label == l:
            return name

In [27]:
get_class_name(0)

'cellphone'

### Get input from webcam and test the 

In [28]:
import cv2
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, (224, 224))
    
    x = np.expand_dims(frame, axis=0)
    
    x = preprocess_input(x)
    
    feature = conv_base.predict(x)
    
    feature = np.reshape(feature, (-1, 7 * 7 * 512))# outout size of vgg conv-flat the outout
    
    predicted = neigh.predict(feature)
        
    text = get_class_name(predicted[0])
    
    cv2.putText(frame,  text, (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), lineType=cv2.LINE_AA)
    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
cap.release()
cv2.destroyAllWindows()     